3種類のアンサンブル学習の効果を小さめのデータセットで確認していきます。

ブレンディング
バギング
スタッキング

# 【問題1】ブレンディング

ブレンディングを実装し、単一モデルより精度があがる例を最低3つ示してください。  
精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを示します。

In [327]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression,Lasso,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing, ensemble, tree, linear_model #機械学習用のライブラリを利用
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

In [328]:
df = pd.read_csv('train.csv')

In [329]:
clf1 = lgb.LGBMRegressor()
clf2 = DecisionTreeRegressor(max_depth=2)
clf3 = ElasticNet()

In [330]:
X =df.loc[:,['GrLivArea','YearBuilt']]
y = df.loc[:,['SalePrice']]

In [331]:
X.describe()

,GrLivArea,YearBuilt
count,1460.000000,1460.000000
mean,1515.463699,1971.267808
std,525.480383,30.202904
min,334.000000,1872.000000
25%,1129.500000,1954.000000
50%,1464.000000,1973.000000
75%,1776.750000,2000.000000
max,5642.000000,2010.000000


In [332]:
y.describe()

,SalePrice
count,1460.000000
mean,180921.195890
std,79442.502883
min,34900.000000
25%,129975.000000
50%,163000.000000
75%,214000.000000
max,755000.000000


In [333]:
num_test = 0.20
sc = StandardScaler()
X=np.array(X)
y=np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=num_test, random_state=23)
sc.fit(X_train)
X_train=sc.transform(X_train)
sc.fit(X_test)
X_test=sc.transform(X_test)

/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [334]:
clf1.fit(X_train,y_train) #線形回帰
y_pred1 = clf1.predict(X_test)
np.log(mean_squared_error(y_test ,y_pred1))

/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


21.3135193002795

In [335]:
clf2.fit(X_train,y_train) #決定木
y_pred2 = clf2.predict(X_test)
np.log(mean_squared_error(y_test, y_pred2))

21.734726609958688

In [336]:
clf3.fit(X_train,y_train) #ElasticNet
y_pred3 = clf3.predict(X_test)
np.log(mean_squared_error(y_test, y_pred3))

21.292246763650407

In [337]:
y_pred_all = (y_pred1 + y_pred2 + y_pred3)/3 #単純に三つの回帰を足している
np.log(mean_squared_error(y_test, y_pred_all))

21.198264315267572

凝ったブレンディング。数値を微調整して良い回帰係数を出している。

In [338]:
y3= 100
best_param={}
for x1 in np.arange(0.1,3.0,0.2):
    for x2 in np.arange(0.1,3.0,0.2):
        for x3 in np.arange(0.1,3.0,0.2):
            y_pred_all = (y_pred1*x1+ y_pred2 * x2+ y_pred3 * x3)/3
            y1 = (np.log(mean_squared_error(y_test, y_pred_all)))
            best_param[y1] = [x1,x2,x3]
            y3 = min(y3,y1)

print(y3)
print(best_param[y3]) #もっとも値の良い重みを出している

21.059275607804445
[1.5000000000000004, 0.1, 1.3000000000000003]


In [339]:
clf1 = lgb.LGBMRegressor(reg_alpha=0.2)
clf2 = DecisionTreeRegressor(max_depth=3)
clf3 = ElasticNet(alpha=1.0, l1_ratio=0.5)

In [340]:
clf1.fit(X_train,y_train) #LinearRegression
y_pred1 = clf1.predict(X_test)
np.log(mean_squared_error(y_test ,y_pred1))

/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


21.31351918788033

In [341]:
clf2.fit(X_train,y_train) #DecisionTreeRegressor
y_pred2 = clf2.predict(X_test)
np.log(mean_squared_error(y_test, y_pred2))

21.534825890573824

In [342]:
clf3.fit(X_train,y_train) #ElasticNet
y_pred3 = clf3.predict(X_test)
np.log(mean_squared_error(y_test, y_pred3))

21.292246763650407

凝ったブレンディング回帰係数の値を微調整

In [343]:
y3= 100
best_param={}
for x1 in np.arange(0.1,3.0,0.2):
    for x2 in np.arange(0.1,3.0,0.2):
        for x3 in np.arange(0.1,3.0,0.2):
            y_pred_all = (y_pred1*x1+ y_pred2 * x2+ y_pred3 * x3)/3
            y1 = (np.log(mean_squared_error(y_test, y_pred_all)))
            best_param[y1] = [x1,x2,x3]
            y3 = min(y3,y1)

print(y3)
print(best_param[y3]) #もっとも値の良い重みを出している

21.059389636690575
[1.5000000000000004, 0.1, 1.3000000000000003]


In [344]:
clf1 = lgb.LGBMRegressor(class_weight='balanced')
clf2 = DecisionTreeRegressor(max_depth=3)
clf3 = ElasticNet(alpha=1.0, l1_ratio=0.5)

In [345]:
clf1.fit(X_train,y_train) #LinearRegression
y_pred1 = clf1.predict(X_test)
np.log(mean_squared_error(y_test ,y_pred1))

/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


21.433809361462743

In [346]:
clf2.fit(X_train,y_train) #DecisionTreeRegressor
y_pred2 = clf2.predict(X_test)
np.log(mean_squared_error(y_test, y_pred2))

21.534825890573824

In [347]:
clf3.fit(X_train,y_train) #ElasticNet
y_pred3 = clf3.predict(X_test)
np.log(mean_squared_error(y_test, y_pred3))

21.292246763650407

In [348]:
y3= 100
best_param={}
for x1 in np.arange(0.1,3.0,0.2):
    for x2 in np.arange(0.1,3.0,0.2):
        for x3 in np.arange(0.1,3.0,0.2):
            y_pred_all = (y_pred1*x1+ y_pred2 * x2+ y_pred3 * x3)/3
            y1 = (np.log(mean_squared_error(y_test, y_pred_all)))
            best_param[y1] = [x1,x2,x3]
            y3 = min(y3,y1)

print(y3) #もっとも良い値
print(best_param[y3]) #ベストな重みをチョイス

21.078362463262167
[1.3000000000000003, 0.1, 1.5000000000000004]


# 【問題2】バギング

In [349]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [350]:
models = {
    'not bagging': DecisionTreeRegressor(),
    'bagging': BaggingRegressor(DecisionTreeRegressor(), n_estimators=100) 
}

In [351]:
for _ in range(20):
    for model_name, model in models.items(): #DecisionTreeRegressorとBaggingRegressor(DecisionTreeRegressor()
        model.fit(X_train, y_train.ravel())
        y_pred = model.predict(X_test)
        s = model_name
        i = np.log(mean_squared_error(y_test, y_pred))
        print('%s の最小２乗誤差 %f' % (s, i))

not bagging の最小２乗誤差 21.660825
bagging の最小２乗誤差 21.222647
not bagging の最小２乗誤差 21.663322
bagging の最小２乗誤差 21.252014
not bagging の最小２乗誤差 21.611018
bagging の最小２乗誤差 21.234368
not bagging の最小２乗誤差 21.657788
bagging の最小２乗誤差 21.253630
not bagging の最小２乗誤差 21.649945
bagging の最小２乗誤差 21.221228
not bagging の最小２乗誤差 21.668314
bagging の最小２乗誤差 21.234013
not bagging の最小２乗誤差 21.655490
bagging の最小２乗誤差 21.213301
not bagging の最小２乗誤差 21.646005
bagging の最小２乗誤差 21.231983
not bagging の最小２乗誤差 21.647643
bagging の最小２乗誤差 21.211676
not bagging の最小２乗誤差 21.670329
bagging の最小２乗誤差 21.222225
not bagging の最小２乗誤差 21.623924
bagging の最小２乗誤差 21.212960
not bagging の最小２乗誤差 21.645259
bagging の最小２乗誤差 21.223080
not bagging の最小２乗誤差 21.654856
bagging の最小２乗誤差 21.245223
not bagging の最小２乗誤差 21.650372
bagging の最小２乗誤差 21.224994
not bagging の最小２乗誤差 21.685797
bagging の最小２乗誤差 21.233445
not bagging の最小２乗誤差 21.678180
bagging の最小２乗誤差 21.222357
not bagging の最小２乗誤差 21.642488
bagging の最小２乗誤差 21.220320
not bagging の最小２乗誤差 21.688321
bagging の最小２乗誤差 21

# 【問題3】スタッキング

In [352]:
df = pd.read_csv('train.csv')

In [353]:
X =df.loc[:,['GrLivArea','YearBuilt']]
y = df.loc[:,['SalePrice']]
X = np.array(X)
y = np.array(y)
print(X.shape)

(1460, 2)


In [354]:
num_test = 0.20
sc = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=num_test, random_state=23)
sc.fit(X_train)
X_train=sc.transform(X_train)
sc.fit(X_test)
X_test=sc.transform(X_test)

/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [355]:
kf = KFold(n_splits=3)
l = [0, 1, 2]
lg = LinearRegression()
dr = DecisionTreeRegressor()

pred_y_lg=[]
pred_y_dr=[]
pred_y_lg_true=[]
pred_y_dr_true=[]
for train, test in kf.split(X_train):

    X_train_train =np.array(X_train[train,:])
    y_train_train =np.array(y_train[train,:])
    X_train_test =np.array(X_train[test,:])
    y_train_test =np.array(y_train[test,:])

    lg.fit(X_train_train,y_train_train)
    dr.fit(X_train_train,y_train_train)
    prelg= lg.predict(X_train_test)
    pred_y_lg.append(prelg)
    predr = dr.predict(X_train_test)
    pred_y_dr.append(predr)
    true_prelg = lg.predict(X_test)
    pred_y_lg_true.append(true_prelg)
    true_predr = dr.predict(X_test)
    pred_y_dr_true.append(true_predr)

pred_y_lg_m = np.concatenate([pred_y_lg[0],pred_y_lg[1],pred_y_lg[2]], axis=0)
pred_y_dr_m = np.concatenate([pred_y_dr[0],pred_y_dr[1],pred_y_dr[2]], axis=0)[:,np.newaxis]
pred_ture_lg = np.mean(pred_y_lg_true,axis=0)
pred_ture_dr = np.mean(pred_y_dr_true,axis=0)[:,np.newaxis]
new_X = np.concatenate([pred_y_lg_m,pred_y_dr_m],axis=1)
new_y = np.concatenate([pred_ture_lg,pred_ture_dr],axis=1)
print("new_X",new_X.shape)

new_X (1168, 2)


In [356]:
kf = KFold(n_splits=3)
l = [0, 1, 2]
sv = SVR(gamma="scale")
lg2 = LinearRegression()

pred_y_sv=[]
pred_y_lg=[]
pred_y_sv_true=[]
pred_y_lg2_true=[]
for train, test in kf.split(new_X):

    X_train_train =np.array(new_X[train,:])
    y_train_train =np.array(y_train[train,:])
    X_train_test =np.array(new_X[test,:])
    y_train_test =np.array(y_train[test,:])
    sv.fit(X_train_train,y_train_train.ravel())
    lg2.fit(X_train_train,y_train_train.ravel())
    presv= sv.predict(X_train_test)
    pred_y_sv.append(presv)
    predlg = lg2.predict(X_train_test)
    pred_y_lg.append(predlg)
    
    true_presv = sv.predict(new_y)
    pred_y_sv_true.append(true_presv)
    true_prelg2 = lg.predict(new_y)
    pred_y_lg2_true.append(true_prelg2)
    
pred_y_sv_m2 = np.concatenate([pred_y_sv[0],pred_y_sv[1],pred_y_sv[2]], axis=0)
pred_y_lg_m2 = np.concatenate([pred_y_lg[0],pred_y_lg[1],pred_y_lg[2]], axis=0)

pred_ture_sv2 = np.mean(pred_y_sv_true,axis=0)
pred_ture_lg2 = np.mean(pred_y_lg2_true,axis=0)

new_X2 = np.concatenate([pred_y_sv_m2[:,np.newaxis],pred_y_lg_m2[:,np.newaxis]],axis=1)
new_y2 = np.concatenate([pred_ture_sv2[:,np.newaxis],pred_ture_lg2],axis=1)

print("new_X2",new_X2.shape)

new_X2 (1168, 2)


In [357]:
# lgr = lgb.LGBMRegressor()
lgr = SVR(gamma="scale")
pred_y_lgr=[]
pred_y_lgr_true=[]

for train, test in kf.split(new_X2):
    X_train_train =np.array(new_X2[train,:])
    y_train_train =np.array(y_train[train,:])
    X_train_test =np.array(new_X2[test,:])
    y_train_test =np.array(y_train[test,:])
    lgr.fit(X_train_train,y_train_train.ravel())
    prelgr= lgr.predict(X_train_test)
    pred_y_lgr.append(prelgr)
    
    prelgr_true= lgr.predict(new_y2)
    pred_y_lgr_true.append(prelgr_true)
last_y = np.concatenate([pred_y_lgr[0],pred_y_lgr[1],pred_y_lgr[2]],axis=0)[:,np.newaxis]
last_pred_y2 = np.mean(pred_y_lgr_true,axis=0)

In [358]:
lg_pred = lg.predict(X_test)
dr_pred = dr.predict(X_test)
sv_pred = sv.predict(X_test)
lg2_pred = lg2.predict(X_test)

print(np.log(mean_squared_error(y_test, lg_pred)))
print(np.log(mean_squared_error(y_test, dr_pred)))
print(np.log(mean_squared_error(y_test, sv_pred)))
print(np.log(mean_squared_error(y_test, lg2_pred)))
print(np.log(mean_squared_error(y_test, last_pred_y2))) #スタッキングで予測したものは５番中3位だった。

21.296870301477977
21.890384942153435
22.376947575240855
24.411374985303237
22.373147113701272


In [359]:
print(last_y.shape)

(1168, 1)
